# Estrazione feature hand-crafted
Dopo aver analizzato e pre-processato i dati ([1_data_preprocessing.ipynb](./1_data_preprocessing.ipynb)) è stata effettuata l'estrazione delle feature.

Le feature hand-crafted rappresentano un insieme di caratteristiche estratte manualmente dai frame appartenenti ai video, mirando a catturare aspetti specifici e distintivi che possono facilitare la successiva fase di classificazione.
Tra le principali feature estratte vi sono quelle legate alla:
- forma,
- tessitura,
- colore,
- movimento.

Sebbene ogni categoria possa offrire un contributo variabile in termini di efficacia, la loro combinazione consente di migliorare significativamente le prestazioni dei sistemi di riconoscimento.

## Import delle librerie

In [ ]:
from handcrafted.app.features.plotter.frames_plotter import (
    plot_frames,
    plot_edge_frames,
    plot_hog_frames,
    plot_haar_frames,
    plot_skin_frames,
    plot_flow_frames,
    plot_lbp_frames,
    plot_color_hist,
)
from handcrafted.app.dataset.dataset import Dataset

## Caricamento del dataset
Di seguito si mostra il caricamento del dataset, e l'estrazione dei frame appartenenti a due diversi video, che verranno utilizzati per mostrare l'estrazione delle features.

In [ ]:
dataset = Dataset("data/WLASL_v0.3.json")

frames_0 = dataset.videos[0].get_frames()
frames_1 = dataset.videos[1].get_frames()

frames = frames_0[10:12] + frames_1[10:12]

Vengono quindi plottati i frame dei due video.

In [ ]:
plot_frames(frames)

## Feature di forma
















### Edge extraction
Tramite la classe [EdgeExtractor](./handcrafted/app/features/extractor/edge_extractor.py) vengono applicate una serie di trasformazioni:
1. Conversione dell'immagine in scala di grigi;
2. Applicazione di un filtro di sfocatura gaussiana;
3. Applicazione di un equalizzatore di istogrammi;
4. Utilizzo dell'operatore morfologico Sobel;
5. Binarizzazione dell'immagine;
6. Operatore di chiusura morfologica.

Di seguito la stampa dei risultati.

In [ ]:
plot_edge_frames(frames)

### Hog detector

La classe [HogExtractor](./handcrafted/app/features/extractor/hog_extractor.py) sfrutta la libreria `skimage` per estrarre le feature in questione.
Oltre all'immagine in scala di grigi, i parametri utilizzati per l'estrazione sono i seguenti:
- `orientations: 9`: il numero di bin dell'istogramma di orientamento;
- `pixels_per_cell: (8,8)`: la dimensione della cella in pixel su cui viene calcolato ogni istogramma gradiente;
- `cells_per_block: (2,2)`: l'area locale su cui verrà normalizzato il conteggio dell'istogramma in una determinata cella;
- `block_norm: "L2-Hys"`: il tipo di normalizzazione da applicare ai blocchi;
- `visualize: True`: oltre alle feature hog, ritorna anche l'immagine dell'istogramma di orientazione risultante (mostrato sotto);
- `transform_sqrt: True`: applica una normalizzazione per ridurre l'effetto delle ombre e migliorare la robustezza rispetto a variazioni di luminosità.

In [ ]:
plot_hog_frames(frames)

## Feature di texture

### Haar-like features

La classe [HaarDetector](./handcrafted/app/features/extractor/haar/haar_detector.py) utilizza il classificatore pre-addestrato `CascadeClassifier` di `opencv` per localizzare i volti dei soggetti.
Esso è basato sul metodo haar e l'algoritmo di Viola-Jones.
Basandosi sull'immagine in scala di grigi viene equalizzato l'istogramma del frame, per migliorare il contrasto dell'immagine.
Si utilizza quindi il classificatore per estrarre le posizioni degli oggetti rilevati.
Siccome `CascadeClassifier` può restituire più posizioni, è stato necessario andare a selezionare quella ritenuta migliore.
Dall'analisi del dataset si è osservato come il volto tende a non cambiare posizione durante il video.
La posizione migliore viene quindi scelta sulla base delle posizioni precedentemente identificate, sfruttando la distanza euclidea.
Le prime posizioni potrebbero però risultare errate sfruttando questa tecnica, in quanto il primo oggetto che viene scelto dipende dalla confidenza che il classificatore assegna alla posizione.
Nei primi dieci frame le posizioni vengono quindi ricalcolate utilizzando la posizione media aggiornata.

In [ ]:
_ = plot_haar_frames(frames)

La pelle dei soggetti è stata estratta utilizzando il classificatore haar in combinazione con l'elaborazione del colore ([SkinExtractor](./handcrafted/app/features/extractor/skin.py)), i passi effettuati sono:
1. trasformazione dello spazio colore in `HSV`;
2. si calcolano i valori medi dei 3 canali;
3. si utilizza un'intorno dei 3 valori medi per estrarre le porzioni dell'immagine nelle quali è presente la pelle del soggetto;
4. viene effettuata infine una operazione di chiusura morfologica.

Di seguito sono mostrati i risultati ottenuti.

In [ ]:
plot_skin_frames(frames_0[10:12])
plot_skin_frames(frames_1[10:12])

### Local Binary Patterns

La classe [LBPExtractor](./handcrafted/app/features/extractor/lbp_extractor.py) sfrutta `skimage` di `opencv` per andare ad estrarre le feature LBP.
I parametri utilizzati per l'estrazione sono i seguenti:
- i frames in scala di grigi;
- `radius: 3`: raggio della circonferenza su cui si trovano i pixel vicini;
- `n_points: 8 * radius`: il numero di vicini considerati;
- `method: uniform`: permette una maggiore robustezza al rumore e riduce la complessità dell'istogramma.

In questo modo viene estratta una matrice in cui ogni valore di pixel rappresenta un valore lbp.
Dalla matrice le feature vengono estratte andando a realizzare un istogramma, come mostrato sotto.
L'istogramma viene inoltre normalizzato garantendo maggiore robustezza rispetto cambiamenti di scala, di illuminazione e traslazione dell'immagine.

In [ ]:
plot_lbp_frames(frames_0[10:12])
plot_lbp_frames(frames_1[10:12])

## Feature colore

### Color histograms

La classe [ColorHistogram](./handcrafted/app/features/extractor/color_histogram_extractor.py) permette di estrarre l'istogramma colore dai frame di un video.
Per fare questo è stata sfruttata la funzione `calcHist` di `opencv` che permette di estrarre o un istogramma per ogni canale dello spazio colore, oppure realizzarne uno multidimensionale che comprende l'intero lo spazio colore.
È possibile andare a normalizzare gli istogrammi, la normalizzazione permette di ottene risultati più robusti rispetto a cambiamenti di scala e traslazioni.

In [ ]:
plot_color_hist(frames_0[10:12], normalize=True)
plot_color_hist(frames_1[10:12], normalize=True)

## Feature di movimento

### Optical flow

La classe [FlowCalculator](./handcrafted/app/features/extractor/flow_calculator.py) sfrutta la libreria `opencv` per calcolare l'optical flow utilizzando il metodo di Gunnar Farnebäck.
I parametri utilizzati per il calcolo sono i seguenti:
- `pyr_scale: 0.3`: indica quanto ridurre l'immagine a ogni livello della piramide;
- `levels: 5`: numero di livelli della piramide;
- `winsize: 10`: dimensione della finestra per le medie ponderate;
- `iterations: 6`: numero di iterazioni per ogni livello della piramide;
- `poly_n: 5`: dimensione della finestra per il filtro polinomiale;
- `poly_sigma: 1.5`: sigma per il filtro polinomiale Gaussiano.

Per disegnare i risultati ottenuti si calcolano la magnitudo e l'angolo dell'optical flow:
- l'angolo, codificato dal colore, indica la direzione del movimento;
- la magnitudo indica l'intensità del movimento e viene rappresentata tramite la luminosità del colore.


In [ ]:
plot_flow_frames(frames_0[10:16])
plot_flow_frames(frames_1[10:16])